# FISH-DS Interactive Visualization with Rerun

This notebook provides an advanced 3D interactive visualization of FISH-DS magnetohydrodynamics (MHD) simulation results using [Rerun](https://rerun.io/). Rerun offers real-time 3D visualization with interactive controls, timeline scrubbing, and multi-dimensional data exploration.

## Features
- **3D Volume Rendering**: Interactive 3D visualization of scalar and vector fields
- **Time Series Animation**: Scrub through simulation timesteps
- **Multi-Variable Display**: Simultaneous visualization of density, velocity, magnetic fields
- **Vector Field Visualization**: 3D arrows for velocity and magnetic field vectors
- **Interactive Controls**: Real-time parameter adjustment and view manipulation
- **Cross-Platform**: Works in Jupyter notebooks and standalone viewer

## Data Structure
- **Input**: FISH-DS binary `.dat` files from simulation output
- **Variables**: Velocity vectors, magnetic fields, density, entropy/pressure
- **Geometry**: 3D grid data with multiple plane segments (XY, XZ, YZ)
- **Temporal**: Multiple timesteps for animation and time series analysis

In [ ]:
# Import Required Libraries
import numpy as np
import struct
import os
import glob
from pathlib import Path
import time
from typing import Dict, List, Optional, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# Rerun for 3D visualization
try:
    import rerun as rr
    print("✅ Rerun imported successfully!")
except ImportError:
    print("❌ Rerun not found. Installing...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rerun-sdk"])
    import rerun as rr
    print("✅ Rerun installed and imported!")

# Additional libraries for data processing
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.cm import get_cmap
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    print("✅ Jupyter widgets available")
except ImportError:
    print("⚠️  Jupyter widgets not available - using basic controls")

print("🚀 All libraries imported successfully!")
print(f"📊 Rerun version: {rr.__version__}")

# Initialize Rerun
rr.init("FISH-DS-MHD-Simulation", spawn=True)
print("🎮 Rerun viewer initialized!")

In [ ]:
# FISH-DS Data Reading Functions
# Adapted from the original vis.ipynb notebook

def read_fortran_record(file_handle, dtype, count=1):
    """
    Read a Fortran unformatted record.
    Fortran unformatted files have record markers before and after each record.
    """
    # Read record size (4 bytes)
    record_size = struct.unpack('i', file_handle.read(4))[0]
    
    # Read the actual data
    if dtype == 'char':
        data = file_handle.read(count).decode('ascii', errors='ignore')
    elif dtype == 'float':
        data = struct.unpack(f'{count}f', file_handle.read(4 * count))
        if count == 1:
            data = data[0]
    elif dtype == 'int':
        data = struct.unpack(f'{count}i', file_handle.read(4 * count))
        if count == 1:
            data = data[0]
    else:
        raise ValueError(f"Unsupported dtype: {dtype}")
    
    # Read trailing record size
    trailing_size = struct.unpack('i', file_handle.read(4))[0]
    
    if record_size != trailing_size:
        print(f"Warning: Record size mismatch: {record_size} != {trailing_size}")
    
    return data

def read_fish_data(filename: str) -> Dict[str, Any]:
    """
    Read FISH-DS binary output file and return structured data.
    
    Returns:
    --------
    data : dict
        Dictionary containing simulation data with keys:
        - 'date': simulation date
        - 'time': simulation time
        - 'dx': grid spacing
        - 'segments': list of data segments (planes)
        - 'domain_overview': reduced resolution domain data
    """
    
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File not found: {filename}")
    
    data = {}
    
    with open(filename, 'rb') as f:
        # Read header information
        data['date'] = read_fortran_record(f, 'char', 8).strip()
        data['time'], data['dx'] = read_fortran_record(f, 'float', 2)
        ns, num, mr = read_fortran_record(f, 'int', 3)
        
        data['num_segments'] = ns
        data['time_step'] = num
        data['processor_rank'] = mr
        
        print(f"📅 Date: {data['date']}")
        print(f"⏰ Time: {data['time']:.6f}")
        print(f"📏 Grid spacing: {data['dx']:.6f}")
        print(f"📊 Segments: {ns}, Time step: {num}, Processor: {mr}")
        
        # Read data segments (3 planes: xy, xz, yz)
        data['segments'] = []
        
        for seg_idx in range(ns):
            segment = {}
            
            # Read segment header
            nv_total = read_fortran_record(f, 'int', 1)
            bounds = read_fortran_record(f, 'int', 6)
            
            segment['num_variables'] = nv_total
            segment['bounds'] = bounds
            imin, imax, jmin, jmax, kmin, kmax = bounds
            
            nx = max(imax - imin + 1, 0)
            ny = max(jmax - jmin + 1, 0)
            nz = max(kmax - kmin + 1, 0)
            
            segment['dimensions'] = (nx, ny, nz)
            
            print(f"  📐 Segment {seg_idx + 1}: {nx}×{ny}×{nz}, {nv_total} variables")
            
            # Read segment data
            if nx * ny * nz > 0:
                total_points = nx * ny * nz * nv_total
                raw_data = read_fortran_record(f, 'float', total_points)
                
                # Reshape data: variables are interleaved for each spatial point
                segment['raw_data'] = np.array(raw_data).reshape((nz, ny, nx, nv_total))
                
                if nv_total >= 6:
                    # Extract velocity components (momentum and magnetic field)
                    segment['velocity'] = segment['raw_data'][:, :, :, :6]
                if nv_total >= 8:
                    # Extract scalar fields (density, entropy)
                    segment['scalars'] = segment['raw_data'][:, :, :, 6:8]
                
            data['segments'].append(segment)
        
        # Try to read domain overview if present
        try:
            nred, i0, j0, k0 = read_fortran_record(f, 'int', 4)
            nv_total = read_fortran_record(f, 'int', 1)
            bounds = read_fortran_record(f, 'int', 6)
            
            imin, imax, jmin, jmax, kmin, kmax = bounds
            nx = max(imax - imin + 1, 0)
            ny = max(jmax - jmin + 1, 0)
            nz = max(kmax - kmin + 1, 0)
            
            if nx * ny * nz > 0:
                total_points = nx * ny * nz * nv_total
                raw_data = read_fortran_record(f, 'float', total_points)
                
                domain_overview = {
                    'reduction_factor': nred,
                    'offset': (i0, j0, k0),
                    'dimensions': (nx, ny, nz),
                    'num_variables': nv_total,
                    'bounds': bounds,
                    'raw_data': np.array(raw_data).reshape((nz, ny, nx, nv_total))
                }
                
                if nv_total >= 6:
                    domain_overview['velocity'] = domain_overview['raw_data'][:, :, :, :6]
                if nv_total >= 8:
                    domain_overview['scalars'] = domain_overview['raw_data'][:, :, :, 6:8]
                
                data['domain_overview'] = domain_overview
                print(f"  🌐 Domain overview: {nx}×{ny}×{nz}, reduction factor: {nred}")
                
        except:
            print("  ⚠️  No domain overview data found")
            data['domain_overview'] = None
    
    return data

def parse_mhd_variables(data_segment: Dict[str, Any]) -> Optional[Dict[str, np.ndarray]]:
    """
    Parse MHD variables from raw data segment.
    
    According to FISH-DS structure:
    - Variables 0-2: momentum density components (vx, vy, vz) * density
    - Variables 3-5: magnetic field components (Bx, By, Bz) / sqrt(4π)
    - Variable 6: density 
    - Variable 7: entropy/internal energy density
    """
    if 'raw_data' not in data_segment:
        return None
    
    raw = data_segment['raw_data']
    nz, ny, nx, nvar = raw.shape
    
    variables = {}
    
    if nvar >= 6:
        # Velocity/momentum components
        variables['momentum_x'] = raw[:, :, :, 0]
        variables['momentum_y'] = raw[:, :, :, 1] 
        variables['momentum_z'] = raw[:, :, :, 2]
        
        # Magnetic field components
        variables['B_x'] = raw[:, :, :, 3]
        variables['B_y'] = raw[:, :, :, 4]
        variables['B_z'] = raw[:, :, :, 5]
    
    if nvar >= 7:
        # Density
        variables['density'] = raw[:, :, :, 6]
        
        # Calculate velocity from momentum and density
        if nvar >= 6:
            # Avoid division by zero
            rho = np.maximum(variables['density'], 1e-10)
            variables['velocity_x'] = variables['momentum_x'] / rho
            variables['velocity_y'] = variables['momentum_y'] / rho
            variables['velocity_z'] = variables['momentum_z'] / rho
            
            # Calculate velocity magnitude
            variables['velocity_magnitude'] = np.sqrt(
                variables['velocity_x']**2 + 
                variables['velocity_y']**2 + 
                variables['velocity_z']**2
            )
            
            # Calculate magnetic field magnitude
            variables['B_magnitude'] = np.sqrt(
                variables['B_x']**2 + 
                variables['B_y']**2 + 
                variables['B_z']**2
            )
    
    if nvar >= 8:
        # Entropy or internal energy density
        variables['entropy'] = raw[:, :, :, 7]
        
        # Calculate pressure (assuming ideal gas)
        if 'density' in variables:
            # For ideal gas: P = (γ-1) * internal_energy_density
            # Assuming γ = 5/3 for monoatomic gas
            gamma = 5.0/3.0
            variables['pressure'] = (gamma - 1) * variables['entropy']
    
    return variables

print("🔧 FISH-DS data reading functions defined successfully!")

In [ ]:
# Rerun Visualization Helper Functions

def create_3d_grid_coordinates(nx: int, ny: int, nz: int, dx: float = 1.0) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Create 3D coordinate grids for visualization."""
    x = np.arange(nx) * dx
    y = np.arange(ny) * dx
    z = np.arange(nz) * dx
    
    # Create 3D coordinate arrays
    Z, Y, X = np.meshgrid(z, y, x, indexing='ij')
    return X, Y, Z

def normalize_scalar_field(field: np.ndarray, min_percentile: float = 1, max_percentile: float = 99) -> np.ndarray:
    """Normalize scalar field to 0-1 range using percentiles to handle outliers."""
    vmin = np.percentile(field, min_percentile)
    vmax = np.percentile(field, max_percentile)
    
    if vmax == vmin:
        return np.zeros_like(field)
    
    normalized = (field - vmin) / (vmax - vmin)
    return np.clip(normalized, 0, 1)

def apply_colormap(scalar_field: np.ndarray, colormap_name: str = 'viridis') -> np.ndarray:
    """Apply colormap to scalar field and return RGBA values."""
    cmap = get_cmap(colormap_name)
    normalized = normalize_scalar_field(scalar_field)
    colors = cmap(normalized)
    
    # Convert to uint8 RGBA format expected by Rerun
    rgba = (colors * 255).astype(np.uint8)
    return rgba

def subsample_vector_field(vx: np.ndarray, vy: np.ndarray, vz: np.ndarray, 
                          X: np.ndarray, Y: np.ndarray, Z: np.ndarray,
                          reduction_factor: int = 4) -> Tuple[np.ndarray, ...]:
    """Subsample vector field for visualization performance."""
    # Subsample the arrays
    vx_sub = vx[::reduction_factor, ::reduction_factor, ::reduction_factor]
    vy_sub = vy[::reduction_factor, ::reduction_factor, ::reduction_factor]
    vz_sub = vz[::reduction_factor, ::reduction_factor, ::reduction_factor]
    
    X_sub = X[::reduction_factor, ::reduction_factor, ::reduction_factor]
    Y_sub = Y[::reduction_factor, ::reduction_factor, ::reduction_factor]
    Z_sub = Z[::reduction_factor, ::reduction_factor, ::reduction_factor]
    
    return vx_sub, vy_sub, vz_sub, X_sub, Y_sub, Z_sub

def log_scalar_field_to_rerun(entity_path: str, scalar_field: np.ndarray, 
                             X: np.ndarray, Y: np.ndarray, Z: np.ndarray,
                             colormap: str = 'viridis', alpha_threshold: float = 0.1):
    """Log a 3D scalar field to Rerun as a point cloud with colors."""
    
    # Flatten arrays
    positions = np.stack([X.flatten(), Y.flatten(), Z.flatten()], axis=1)
    values = scalar_field.flatten()
    
    # Apply colormap
    colors = apply_colormap(values, colormap)
    
    # Filter points based on alpha threshold (remove low-value points for performance)
    normalized_values = normalize_scalar_field(values)
    mask = normalized_values > alpha_threshold
    
    if np.any(mask):
        positions_filtered = positions[mask]
        colors_filtered = colors[mask]
        
        # Log to Rerun
        rr.log(
            entity_path,
            rr.Points3D(
                positions=positions_filtered,
                colors=colors_filtered[:, :3],  # RGB only
                radii=0.5  # Adjust point size as needed
            )
        )
    else:
        print(f"⚠️  No points above threshold for {entity_path}")

def log_vector_field_to_rerun(entity_path: str, vx: np.ndarray, vy: np.ndarray, vz: np.ndarray,
                             X: np.ndarray, Y: np.ndarray, Z: np.ndarray,
                             scale_factor: float = 1.0, color: Tuple[int, int, int] = (255, 255, 255),
                             uniform_length: bool = True, colormap: str = 'viridis'):
    """Log a 3D vector field to Rerun as arrows with optional uniform length and magnitude-based coloring."""
    
    # Subsample for performance
    vx_sub, vy_sub, vz_sub, X_sub, Y_sub, Z_sub = subsample_vector_field(
        vx, vy, vz, X, Y, Z, reduction_factor=4
    )
    
    # Create arrow origins and vectors
    origins = np.stack([X_sub.flatten(), Y_sub.flatten(), Z_sub.flatten()], axis=1)
    vectors = np.stack([vx_sub.flatten(), vy_sub.flatten(), vz_sub.flatten()], axis=1)
    
    # Calculate magnitudes
    magnitudes = np.linalg.norm(vectors, axis=1)
    
    # Filter out zero vectors
    mask = magnitudes > 1e-10
    
    if np.any(mask):
        origins_filtered = origins[mask]
        vectors_filtered = vectors[mask]
        magnitudes_filtered = magnitudes[mask]
        
        if uniform_length:
            # Normalize vectors to unit length and scale
            unit_vectors = vectors_filtered / magnitudes_filtered[:, np.newaxis]
            display_vectors = unit_vectors * scale_factor
            
            # Apply colormap based on magnitude
            magnitude_colors = apply_colormap(magnitudes_filtered, colormap)
            arrow_colors = magnitude_colors[:, :3]  # RGB only
        else:
            # Use original scaled vectors
            display_vectors = vectors_filtered * scale_factor
            # Use uniform color
            arrow_colors = np.tile(color, (len(vectors_filtered), 1))
        
        # Log to Rerun
        rr.log(
            entity_path,
            rr.Arrows3D(
                origins=origins_filtered,
                vectors=display_vectors,
                colors=arrow_colors
            )
        )
    else:
        print(f"⚠️  No non-zero vectors for {entity_path}")

print("🎨 Rerun visualization helper functions defined successfully!")

In [ ]:
# Main FISH-DS Rerun Visualizer Class

class FishDSRerunVisualizer:
    """
    Interactive 3D visualizer for FISH-DS simulation data using Rerun.
    """
    
    def __init__(self, data_directory: str = "./data", lazy_load: bool = True):
        """Initialize the visualizer with data directory.
        
        Args:
            data_directory: Path to data directory containing proc*/*.dat files
            lazy_load: If True, only discover files without loading them immediately
        """
        self.data_dir = Path(data_directory)
        self.dat_files = []
        self.current_data = None
        self.time_steps = []
        self._files_discovered = False
        
        # Visualization parameters
        self.show_density = True
        self.show_velocity = True
        self.show_magnetic = True
        self.show_pressure = False
        self.vector_scale = 1.0
        self.alpha_threshold = 0.1
        
        # Color schemes
        self.density_colormap = 'viridis'
        self.velocity_colormap = 'plasma'
        self.magnetic_colormap = 'coolwarm'
        self.pressure_colormap = 'hot'
        
        # Discover files based on lazy_load setting
        if not lazy_load:
            self.discover_data_files()
        else:
            print(f"🏃‍♂️ Fast initialization enabled. Call viz.discover_data_files() to scan for data files.")
    
    def discover_data_files(self):
        """Find all available .dat files in the data directory."""
        if self._files_discovered:
            print("📁 Files already discovered. Use refresh=True to rescan.")
            return
            
        print(f"🔍 Searching for data files in: {self.data_dir.absolute()}")
        
        # Check if data directory exists
        if not self.data_dir.exists():
            # Try alternative paths
            alt_paths = [Path("../data"), Path("./fish-basic/data"), Path(".")]
            for alt_path in alt_paths:
                if alt_path.exists():
                    self.data_dir = alt_path
                    print(f"  📁 Found data directory: {self.data_dir.absolute()}")
                    break
            else:
                print("❌ Data directory not found!")
                return
        
        # Find .dat files in processor subdirectories - optimized scanning
        self.dat_files = []
        total_files = 0
        
        try:
            # Use glob pattern for faster file discovery
            dat_pattern = self.data_dir / "proc*" / "*.dat"
            all_dat_files = list(self.data_dir.glob("proc*//*.dat"))
            
            if not all_dat_files:
                print("❌ No .dat files found in processor directories")
                return
            
            # Group by processor directory for reporting
            proc_files = {}
            for file_path in all_dat_files:
                proc_name = file_path.parent.name
                if proc_name not in proc_files:
                    proc_files[proc_name] = []
                proc_files[proc_name].append(file_path)
            
            # Report findings and collect files
            for proc_name, files in sorted(proc_files.items()):
                self.dat_files.extend(files)
                total_files += len(files)
                print(f"  📄 Found {len(files)} files in {proc_name}")
            
        except Exception as e:
            print(f"❌ Error scanning for files: {e}")
            return
        
        # Sort files for consistent ordering
        self.dat_files.sort()
        
        # Extract time steps from filenames - optimized parsing
        self.time_steps = []
        for f in self.dat_files:
            try:
                # Optimized filename parsing
                stem_parts = f.stem.split('_')
                if len(stem_parts) >= 2 and stem_parts[1].isdigit():
                    time_step = int(stem_parts[1])
                    self.time_steps.append(time_step)
                else:
                    self.time_steps.append(0)
            except (ValueError, IndexError):
                self.time_steps.append(0)
        
        print(f"📊 Total files found: {total_files}")
        if self.dat_files:
            print(f"⏰ Time step range: {min(self.time_steps)} - {max(self.time_steps)}")
            self._files_discovered = True
        else:
            print("❌ No data files discovered")
    
    def refresh_file_list(self):
        """Force refresh of the file list."""
        self._files_discovered = False
        self.dat_files = []
        self.time_steps = []
        self.discover_data_files()
    
    def load_timestep(self, timestep_index: int = 0) -> bool:
        """Load data for a specific timestep."""
        # Ensure files are discovered
        if not self._files_discovered:
            self.discover_data_files()
            
        if not self.dat_files or timestep_index >= len(self.dat_files):
            print(f"❌ Invalid timestep index: {timestep_index} (available: 0-{len(self.dat_files)-1})")
            return False
        
        filename = self.dat_files[timestep_index]
        print(f"📖 Loading: {filename.name}")
        
        try:
            self.current_data = read_fish_data(str(filename))
            return True
        except Exception as e:
            print(f"❌ Error loading {filename}: {e}")
            return False
    
    def visualize_segment(self, segment_index: int = 0, timestep_index: int = 0):
        """Visualize a specific data segment."""
        if not self.load_timestep(timestep_index):
            return
        
        if segment_index >= len(self.current_data['segments']):
            print(f"❌ Invalid segment index: {segment_index} (available: 0-{len(self.current_data['segments'])-1})")
            return
        
        segment = self.current_data['segments'][segment_index]
        variables = parse_mhd_variables(segment)
        
        if not variables:
            print("❌ No variables found in segment")
            return
        
        # Get dimensions and create coordinate grids
        nz, ny, nx = segment['dimensions']
        dx = self.current_data['dx']
        
        print(f"🎯 Visualizing segment {segment_index + 1}: {nx}×{ny}×{nz}")
        
        X, Y, Z = create_3d_grid_coordinates(nx, ny, nz, dx)
        
        # Set timeline
        time_ns = int(self.current_data['time'] * 1e9)  # Convert to nanoseconds
        rr.set_time_nanos("simulation_time", time_ns)
        
        # Log coordinate system
        rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Y_UP, static=True)
        
        # Visualize scalar fields
        if self.show_density and 'density' in variables:
            log_scalar_field_to_rerun(
                "world/density",
                variables['density'], X, Y, Z,
                colormap=self.density_colormap,
                alpha_threshold=self.alpha_threshold
            )
        
        if self.show_pressure and 'pressure' in variables:
            log_scalar_field_to_rerun(
                "world/pressure",
                variables['pressure'], X, Y, Z,
                colormap=self.pressure_colormap,
                alpha_threshold=self.alpha_threshold
            )
        
        # Visualize velocity field
        if self.show_velocity and all(f'velocity_{c}' in variables for c in ['x', 'y', 'z']):
            log_vector_field_to_rerun(
                "world/velocity_field",
                variables['velocity_x'], variables['velocity_y'], variables['velocity_z'],
                X, Y, Z,
                scale_factor=self.vector_scale,
                uniform_length=True,
                colormap='plasma'  # Hot colors for velocity magnitude
            )
        
        # Visualize magnetic field
        if self.show_magnetic and all(f'B_{c}' in variables for c in ['x', 'y', 'z']):
            log_vector_field_to_rerun(
                "world/magnetic_field",
                variables['B_x'], variables['B_y'], variables['B_z'],
                X, Y, Z,
                scale_factor=self.vector_scale * 0.7,  # Slightly smaller than velocity
                uniform_length=True,
                colormap='coolwarm'  # Blue-red for magnetic field magnitude
            )
        
        print(f"✅ Visualization complete for timestep {timestep_index}")
    
    def visualize_domain_overview(self, timestep_index: int = 0):
        """Visualize the domain overview data if available."""
        if not self.load_timestep(timestep_index):
            return
        
        if not self.current_data['domain_overview']:
            print("❌ No domain overview data available")
            return
        
        dov = self.current_data['domain_overview']
        variables = parse_mhd_variables(dov)
        
        if not variables:
            print("❌ No variables found in domain overview")
            return
        
        # Get dimensions and create coordinate grids
        nz, ny, nx = dov['dimensions']
        dx = self.current_data['dx'] * dov['reduction_factor']
        
        print(f"🌐 Visualizing domain overview: {nx}×{ny}×{nz}")
        
        X, Y, Z = create_3d_grid_coordinates(nx, ny, nz, dx)
        
        # Set timeline
        time_ns = int(self.current_data['time'] * 1e9)
        rr.set_time_nanos("simulation_time", time_ns)
        
        # Log coordinate system
        rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Y_UP, static=True)
        
        # Visualize overview data (same as segment but with different path)
        if self.show_density and 'density' in variables:
            log_scalar_field_to_rerun(
                "world/domain_overview/density",
                variables['density'], X, Y, Z,
                colormap=self.density_colormap,
                alpha_threshold=self.alpha_threshold * 0.5  # Lower threshold for overview
            )
        
        if self.show_velocity and all(f'velocity_{c}' in variables for c in ['x', 'y', 'z']):
            log_vector_field_to_rerun(
                "world/domain_overview/velocity_field",
                variables['velocity_x'], variables['velocity_y'], variables['velocity_z'],
                X, Y, Z,
                scale_factor=self.vector_scale,
                uniform_length=True,
                colormap='plasma'
            )
        
        if self.show_magnetic and all(f'B_{c}' in variables for c in ['x', 'y', 'z']):
            log_vector_field_to_rerun(
                "world/domain_overview/magnetic_field",
                variables['B_x'], variables['B_y'], variables['B_z'],
                X, Y, Z,
                scale_factor=self.vector_scale * 0.7,
                uniform_length=True,
                colormap='coolwarm'
            )
        
        print(f"✅ Domain overview visualization complete for timestep {timestep_index}")
    
    def animate_time_series(self, max_timesteps: int = 10, delay: float = 0.5):
        """Create an animation through multiple timesteps."""
        # Ensure files are discovered
        if not self._files_discovered:
            self.discover_data_files()
            
        if not self.dat_files:
            print("❌ No data files available for animation")
            return
            
        print(f"🎬 Creating animation for {min(max_timesteps, len(self.dat_files))} timesteps")
        
        for i in range(min(max_timesteps, len(self.dat_files))):
            print(f"\n⏯️  Frame {i+1}/{min(max_timesteps, len(self.dat_files))}")
            
            # Try domain overview first, fall back to first segment
            if self.current_data and self.current_data.get('domain_overview'):
                self.visualize_domain_overview(i)
            else:
                self.visualize_segment(0, i)
            
            time.sleep(delay)
        
        print("\n🎉 Animation complete!")
    
    def configure_visualization(self, show_density: bool = True, show_velocity: bool = True,
                             show_magnetic: bool = True, show_pressure: bool = False,
                             vector_scale: float = 1.0, alpha_threshold: float = 0.1):
        """Configure visualization parameters."""
        self.show_density = show_density
        self.show_velocity = show_velocity
        self.show_magnetic = show_magnetic
        self.show_pressure = show_pressure
        self.vector_scale = vector_scale
        self.alpha_threshold = alpha_threshold
        
        print("⚙️  Visualization configuration updated:")
        print(f"   📊 Density: {show_density}")
        print(f"   🌊 Velocity: {show_velocity}")
        print(f"   🧲 Magnetic: {show_magnetic}")
        print(f"   🌡️  Pressure: {show_pressure}")
        print(f"   📏 Vector scale: {vector_scale}")
        print(f"   👁️  Alpha threshold: {alpha_threshold}")
    
    def get_file_info(self):
        """Get information about discovered files."""
        if not self._files_discovered:
            print("📁 Files not yet discovered. Call discover_data_files() first.")
            return
            
        print(f"📊 File Information:")
        print(f"   Total files: {len(self.dat_files)}")
        print(f"   Time steps: {min(self.time_steps)} to {max(self.time_steps)}")
        print(f"   Data directory: {self.data_dir.absolute()}")
        
        if len(self.dat_files) > 0:
            print(f"   First file: {self.dat_files[0].name}")
            print(f"   Last file: {self.dat_files[-1].name}")

print("🎮 FishDSRerunVisualizer class defined successfully!")
print("💡 Performance improvements:")
print("   - Lazy loading: Files are only scanned when needed")
print("   - Optimized file discovery using glob patterns")
print("   - Better error handling and user feedback")
print("   - Fast initialization option")

In [ ]:
# Initialize FISH-DS Rerun Visualizer (Optimized)

# Create the visualizer instance with fast initialization
print("🚀 Creating FISH-DS Rerun Visualizer...")
viz = FishDSRerunVisualizer(data_directory="./data", lazy_load=True)

# Discover data files (this step can be skipped for even faster startup)
print("\n🔍 Discovering data files...")
viz.discover_data_files()

# Check if data was found
if not viz.dat_files:
    print("\n❌ No data files found! Please ensure you have FISH-DS output files in:")
    print("   ./data/proc*/*.dat")
    print("\n💡 If your data is elsewhere, try:")
    print("   viz = FishDSRerunVisualizer('path/to/your/data')")
    print("   viz.discover_data_files()")
else:
    print(f"\n✅ Successfully initialized with {len(viz.dat_files)} data files")
    print(f"📅 Time steps available: {min(viz.time_steps)} to {max(viz.time_steps)}")
    
    # Configure visualization settings
    viz.configure_visualization(
        show_density=True,      # Show density as colored points
        show_velocity=True,     # Show velocity vectors (green arrows)
        show_magnetic=True,     # Show magnetic field vectors (red arrows)
        show_pressure=False,    # Don't show pressure by default
        vector_scale=2.0,       # Scale factor for vector arrows
        alpha_threshold=0.2     # Only show points above this normalized value
    )
    
    print("\n🎯 Ready for visualization!")
    print("\n📋 Quick start commands:")
    print("   viz.visualize_segment(0, 0)                           # First segment, first timestep")
    print("   viz.visualize_domain_overview(0)                      # Domain overview, first timestep")
    print("   viz.animate_time_series(max_timesteps=3, delay=1.0)   # Short animation")
    print("\n🔧 Utility commands:")
    print("   viz.get_file_info()                                   # Show file information")
    print("   viz.refresh_file_list()                               # Rescan for new files")
    print("   viz.configure_visualization(...)                      # Change settings")

In [ ]:
viz = FishDSRerunVisualizer(data_directory="./data", lazy_load=True)
viz.configure_visualization(vector_scale=10)
viz.animate_time_series(100, delay=0)

In [ ]:
# Create visualizer with enhanced vector settings
viz_enhanced = FishDSRerunVisualizer(data_directory="./data", lazy_load=True)

# Configure for optimal vector visualization
viz_enhanced.configure_visualization(
    show_density=True,        # Show density as background
    show_velocity=True,       # Show velocity vectors
    show_magnetic=True,       # Show magnetic field vectors
    show_pressure=False,      # Skip pressure for cleaner view
    vector_scale=1.5,         # Good arrow length
    alpha_threshold=0       # Only show significant density points
)